In [10]:
import math
import pandas as pd
import numpy as np

def population_in_range(input_path = 'test.pkl', output_path = 'test.tif', radius = 1):
    # function that calculates the number of inhabitants within the range specified in "radius". The variable "radius" should be an integer value.
    
    # --- define help functions ---
    def weighting(radius = 1):

        width = 0.5

        resolution = 20
        step = 1/resolution

        rng = range(-radius-1,radius+2)
        df = pd.DataFrame(index=rng, columns=rng)

        for x in rng:
            for y in rng:
                count = 0
                for x0 in np.arange(-0.5, 0.5, step):
                    for y0 in np.arange(-0.5, 0.5, step):
                        for x1 in np.arange(x-0.5, x+0.5, step):
                            for y1 in np.arange(y-0.5, y+0.5, step):
                                distance = math.sqrt((x1-x0)**2 + (y1-y0)**2)
                                if (radius - width <= distance) & (distance < radius + width):
                                    count += 1 / resolution**4
                df.loc[x, y] = count

        return(df)
    
    def shift_from(y, x, df):
        def from_x(x, df):
            if x < 0:
                return(df.columns[:x])
            else:
                return(df.columns[x:])

        def from_y(y, df):
            if y < 0:
                return(df.index[:y])
            else:
                return(df.index[y:])

        return(df.loc[from_y(y, df), from_x(x, df)])

    def to_x(x, df):
        if x <= 0:
            return(df.columns[-x:])
        else:
            return(df.columns[:-x])

    def to_y(y, df):
        if y <= 0:
            return(df.index[-y:])
        else:
            return(df.index[:-y])
        
    def habitants_in_range(dfi, distance = 1):
        weights = weighting(distance)

        dfo = pd.DataFrame(0, index = dfi.index, columns = dfi.columns)

        for x in weights.columns:
            for y in weights.index:
                dfo.loc[to_y(y, dfo), to_x(x, dfo)] += shift_from(y, x, dfi).values * weights.loc[y, x]

        return(dfo)

    # --- end of help functions ---
    
    raw = pd.read_pickle(input_path)
    
    df = habitants_in_range(raw['Population'].unstack().sort_index().sort_index(axis=1), radius)
    
    # --- TO DO ---
    # df still needs to be converted into a .tif (probably using rasterio)
    # safe df as tiff at this location: output_path
    